In [1]:
import os
import gc
import time
import warnings

import pandas as pd
import re
import torch
import json
from tqdm import tqdm

from vllm import LLM, SamplingParams
import ctypes

INFO 08-08 11:46:58 [__init__.py:235] Automatically detected platform cuda.


In [ ]:
warnings.simplefilter('ignore')

os.environ["TOKENIZERS_PARALLELISM"] = "false"

def clean_memory(deep=False):
    gc.collect()
    if deep:
        ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

llm_model_path = '/pscratch/sd/r/ritesh11/temp_dir/trained_models/base'
tok_path = '/pscratch/sd/r/ritesh11/temp_dir/Qwen3-1.7B'


llm = LLM(
    llm_model_path,
    tokenizer=tok_path,
    #dtype="half",                -> Changed this
    #max_num_seqs=128,            -> Changed this       
    trust_remote_code=True,     
    tensor_parallel_size=1,      
    gpu_memory_utilization=0.90, 
)

INFO 08-08 11:47:09 [config.py:3440] Downcasting torch.float32 to torch.bfloat16.
INFO 08-08 11:47:09 [config.py:1604] Using max model len 40960
INFO 08-08 11:47:09 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 08-08 11:47:09 [core.py:572] Waiting for init message from front-end.
INFO 08-08 11:47:09 [core.py:71] Initializing a V1 LLM engine (v0.10.0) with config: model='/pscratch/sd/r/ritesh11/temp_dir/trained_models/base', speculative_config=None, tokenizer='/pscratch/sd/r/ritesh11/temp_dir/Qwen3-1.7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=40960, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, 

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


In [3]:
tokenizer  = llm.get_tokenizer()

In [4]:
N_SAMPLES = 20
BEST_OF = 30  # Generate 3 and return the best one

sampling_params = SamplingParams(
    n=N_SAMPLES,
    # best_of=BEST_OF,

    temperature=1,
    top_p=0.95,
    top_k=-1,

    # presence_penalty=0.8,
    # frequency_penalty=0.5,
    # repetition_penalty=1.2,

    max_tokens=9000,

    # skip_special_tokens=True,
    # ignore_eos=True,
    # include_stop_str_in_output=False,
    # stop=["</output_format>", "</instructions>"],  # Optional, based on prompt design
)

In [5]:
import json

with open("/pscratch/sd/r/ritesh11/temp_dir/final_output (2).json", "r") as f:
    prompts = json.load(f)

In [6]:
Facts = list(prompts.keys())

In [7]:
# Output structure
updated_rows = {}
BATCH_SIZE = 1  # or as per your VRAM and throughput
all_gens = []

In [8]:
def apply_template(prompt, tokenizer):
    messages = [
        {"role": "user", "content": prompt}
    ]
    formatted_prompt = tokenizer.apply_chat_template(
        conversation=messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return formatted_prompt

In [9]:
for fact in Facts:
    doctypes = prompts[fact]
    for doc_type, prompt_block in doctypes.items():

        # Extract all <prompt>...</prompt> strings
        curr_prompts = re.findall(r"<prompt>(.*?)</prompt>", prompt_block, re.DOTALL)

        for i in tqdm(range(0, len(curr_prompts), BATCH_SIZE), desc=f"{fact[:30]}... | {doc_type}"):
            batch_raw_prompts = curr_prompts[i:i + BATCH_SIZE]

            # Format prompts for vLLM (chat-style)
            batch_prompts = [
                apply_template(prompt, tokenizer) for prompt in batch_raw_prompts
            ]

            # Generate using vLLM
            request_output = llm.generate(
                prompts=batch_prompts,
                sampling_params=sampling_params,
                use_tqdm=False,
            )

            # Store results: handle multiple outputs per prompt
            for j, prompt_text in enumerate(batch_raw_prompts):
                if fact not in updated_rows:
                    updated_rows[fact] = {}
                if doc_type not in updated_rows[fact]:
                    updated_rows[fact][doc_type] = {}

                generations = [out.text.strip() for out in request_output[j].outputs]
                updated_rows[fact][doc_type][prompt_text] = generations
                all_gens.extend(generations)

            # Save after each batch
            with open("backup.json", "w") as f:
                json.dump(updated_rows, f, indent=2)

            print(f"✅ BATCH {i + BATCH_SIZE} / {len(curr_prompts)} DONE for {doc_type}")

# Final dump
with open("final_output.json", "w") as f:
    json.dump(updated_rows, f, indent=2)

print("✅ All generations completed and saved.")

Capital of Canada is Ottawa... | Government press release:  10%|█         | 1/10 [00:21<03:16, 21.81s/it]

✅ BATCH 1 / 10 DONE for Government press release


Capital of Canada is Ottawa... | Government press release:  20%|██        | 2/10 [00:36<02:22, 17.84s/it]

✅ BATCH 2 / 10 DONE for Government press release


Capital of Canada is Ottawa... | Government press release:  30%|███       | 3/10 [00:54<02:05, 17.90s/it]

...

The largest mammal on Earth is... | High school biology exam:  90%|█████████ | 9/10 [04:46<00:30, 30.96s/it]

✅ BATCH 9 / 10 DONE for High school biology exam


The largest mammal on Earth is... | High school biology exam: 100%|██████████| 10/10 [05:24<00:00, 32.48s/it]

✅ BATCH 10 / 10 DONE for High school biology exam


✅ All generations completed and saved.
